In [1]:
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat

In [18]:
val spark = SparkSession
                .builder
                .master("local[*]")
                .appName("SparkSQL")
                .config("spark.cassandra.connection.host", "127.0.0.1")
                .getOrCreate()

import spark.implicits._

// Reading the file from disk
val conf = new Configuration
conf.set("textinputformat.record.delimiter", "WARC/1.0")
val dataset = spark
                .sparkContext
                .newAPIHadoopFile(
                    "file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/WET/*.wet",
//                     "../data/WET/CC-MAIN-20180918130631-20180918150631-00000.warc.wet",
                    classOf[TextInputFormat], 
                    classOf[LongWritable], 
                    classOf[Text], 
                    conf
                )

import org.apache.spark.sql.functions._


spark = org.apache.spark.sql.SparkSession@762611da
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
dataset = file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/WET/*.wet NewHadoopRDD[52] at newAPIHadoopFile at <console>:59


file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/WET/*.wet NewHadoopRDD[52] at newAPIHadoopFile at <console>:59

In [19]:
var tempdata = dataset.map(x=>x._2.toString).toDF();

// Filter empty stuff
tempdata = tempdata.filter(length($"value") > 0)

// Split header
tempdata = tempdata.withColumn("header", 
                        split($"value", "\r\n\r\n").getItem(0)
                    )
                    .withColumn("text", 
                        split($"value", "\r\n\r\n").getItem(1)
                    )
                    .drop("value")

// preparse header
tempdata = tempdata.withColumn("header",
                        split($"header", "\r\n")
                    )
//                     .withColum("header",
//                         filter($"header")
//                     )

tempdata.printSchema()

tempdata = tempdata.withColumn("warc-type",
                    ($"header").getItem(1)
                )
                .filter($"warc-type" === "WARC-Type: conversion")
                .drop("warc-type")


// Extract uri
tempdata = tempdata.withColumn("uri",
                        ($"header").getItem(2)
                    )
                    .withColumn("uri",
                        split($"uri", ": ").getItem(1)
                    )
                    .drop("header")

// Extract domain
val regexpr = """[^((https?):\/\/)]((www|www1)\.)?([\w-\.]+)"""
tempdata = tempdata.withColumn("domain",
                    regexp_extract($"uri", regexpr, 0)
                )
                
// println(warctype.first.getString(0))
// tempdata.show();

val pageInfoDF = tempdata

root
 |-- header: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text: string (nullable = true)

+--------------------+--------------------+--------------------+
|                text|                 uri|              domain|
+--------------------+--------------------+--------------------+
|Новости 0-50.ru |...|http://0-50.ru/ne...|             0-50.ru|
|تبلیغات	
مسخره - ...|http://000elham00...|000elham00.mihanb...|
|맛집? 많은분 들이 이곳을 이용...|http://0105335908...|01053359081.kte11...|
|認知症ケアマニュアル: アルツハイ...|http://01ninchisy...|01ninchisyou.sees...|
|正在播放 5-鬼畜老师强暴学生妹[...|http://01xxxx.com...|          01xxxx.com|
|皮肤病 / 荨麻疹_重庆皮肤病医院...|http://023pifu.co...|         023pifu.com|
|Amicus Brief FAQs...|http://03e98e7.ne...|03e98e7.netsolhos...|
|2017款宝马X5 xDrive3...|http://0411auto.c...|        0411auto.com|
|こら！ぼたち！ ダッシュするときも...|http://0412hitomi...|0412hitomi.blog.f...|
|網路聊天聯盟
回 首 頁 │ 點數...|http://080ek21.co...|         080ek21.com|
|Мангал | Леко мно...|http:/

tempdata = [text: string, uri: string ... 1 more field]
tempdata = [text: string, uri: string ... 1 more field]
tempdata = [text: string, uri: string ... 1 more field]
tempdata = [text: string, uri: string ... 1 more field]
tempdata = [text: string, uri: string ... 1 more field]
tempdata = [text: string, uri: string ... 1 more field]
regexpr = [^((https?):\/\/)]((www|www1)\.)?([\w-\.]+)
tempdata = [text: string, uri: string ... 1 more field]
pageInfoDF = [text: string, uri: string ... 1 more field]


[text: string, uri: string ... 1 more field]

In [20]:
import org.apache.spark.sql.functions._ 

val extractKeyWords = udf[Array[(String, Int)], String](input => {
    val stopWords = Set("i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now")
    
    input
      .split("[\\s|\n]")
      .map(s => s.replaceAll("""[^\x00-\x7F]""", "")) // We only manage ascii chars
      .map(s => s.replaceAll("""^\p{Punct}*$""", "")) // No words with only punctuation
      .map(s => s.replaceAll("""\p{Punct}$""", ""))   // No trailing punctuations
      .filter(_ != "")                                // No empty words
      .map(_.toLowerCase)
      .filter(!stopWords.contains(_))                 // Remove stop-words
      .groupBy(identity).mapValues(_.size).toArray
      .sortBy(- _._2)
})


val keywordDF = pageInfoDF.withColumn(
    "keywords",
    extractKeyWords(col("text"))
  )

keywordDF
//     .select("keywords")
    .show()


+--------------------+--------------------+--------------------+--------------------+
|                text|                 uri|              domain|            keywords|
+--------------------+--------------------+--------------------+--------------------+
|Новости 0-50.ru |...|http://0-50.ru/ne...|             0-50.ru|[[2, 13], [1:1, 4...|
|تبلیغات	
مسخره - ...|http://000elham00...|000elham00.mihanb...|[[1396, 15], [139...|
|맛집? 많은분 들이 이곳을 이용...|http://0105335908...|01053359081.kte11...|[[01053359081.kte...|
|認知症ケアマニュアル: アルツハイ...|http://01ninchisy...|01ninchisyou.sees...|[[(7, 2], [(5, 2]...|
|正在播放 5-鬼畜老师强暴学生妹[...|http://01xxxx.com...|          01xxxx.com|[[2017-02-16, 13]...|
|皮肤病 / 荨麻疹_重庆皮肤病医院...|http://023pifu.co...|         023pifu.com|[[hospital, 3], [...|
|Amicus Brief FAQs...|http://03e98e7.ne...|03e98e7.netsolhos...|[[appellate, 4], ...|
|2017款宝马X5 xDrive3...|http://0411auto.c...|        0411auto.com|[[xdrive35i, 3], ...|
|こら！ぼたち！ ダッシュするときも...|http://0412hitomi...|0412hitomi.

extractKeyWords = UserDefinedFunction(<function1>,ArrayType(StructType(StructField(_1,StringType,true), StructField(_2,IntegerType,false)),true),Some(List(StringType)))
keywordDF = [text: string, uri: string ... 2 more fields]


[text: string, uri: string ... 2 more fields]

In [27]:
// create one list with all keywords
// keywords distinct = 2017741 (.distinct.count)
// keywords total = 15 000 000
// total_nr_records = 39653
val keywordsExpandedDF = keywordDF
                            .withColumn("keywords", explode(($"keywords")))
                            .withColumn("keyword", $"keywords._1")
                            .withColumn("keyword_weight", $"keywords._2")
                            .drop("keywords")

keywordsExpandedDF.show


+--------------------+--------------------+-------+---------+--------------+
|                text|                 uri| domain|  keyword|keyword_weight|
+--------------------+--------------------+-------+---------+--------------+
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|        2|            13|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|      1:1|             4|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|       15|             3|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|      1:0|             3|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|      0:1|             3|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|      1:2|             3|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|     2017|             3|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|      2:0|             2|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|       22|             2|
|Новости 0-50.ru |...|http://0-50.ru/ne...|0-50.ru|    -2114|             2|

keywordsExpandedDF = [text: string, uri: string ... 3 more fields]


[text: string, uri: string ... 3 more fields]

In [25]:
var reverseIndex = keywordsExpandedDF
                        .groupBy($"keyword")
                        .agg(
                            collect_list($"uri").alias("uri_list"), 
                            collect_list($"keyword_weight").alias("keyword_weights")
                            ) 
// We can probably collect as struct collect_list(struct($"uri", $"keyword_weight")).as("set")
// This might however lead to problems: https://stackoverflow.com/questions/31864744/spark-dataframes-groupby-into-list
                        
// Showing here takes 1 minute
// reducedKeywords.show() 


reverseIndex = [keyword: string, uri_list: array<string> ... 1 more field]


[keyword: string, uri_list: array<string> ... 1 more field]

In [1]:
// Import and connect to cassandra
import org.apache.spark.sql.cassandra._
import com.datastax.spark.connector._
import com.datastax.driver.core.{Session, Cluster, Host, Metadata}
import com.datastax.spark.connector.cql.CassandraConnectorConf
import com.datastax.spark.connector.rdd.ReadConf

val cluster = Cluster.builder().addContactPoint("127.0.0.1").build()
val session = cluster.connect()

Name: Compile Error
Message: <console>:26: error: object cassandra is not a member of package org.apache.spark.sql
       import org.apache.spark.sql.cassandra._
                                   ^
<console>:27: error: object datastax is not a member of package com
       import com.datastax.spark.connector._
                  ^
<console>:28: error: object datastax is not a member of package com
       import com.datastax.driver.core.{Session, Cluster, Host, Metadata}
                  ^
<console>:37: error: not found: value Cluster
       val cluster = Cluster.builder().addContactPoint("127.0.0.1").build()
                     ^
<console>:29: error: object datastax is not a member of package com
       import com.datastax.spark.connector.cql.CassandraConnectorConf
                  ^
<console>:30: error: object datastax is not a member of package com
       import com.datastax.spark.connector.rdd.ReadConf
                  ^

StackTrace: 

In [73]:
// Saving to cassandra. This was too complicated with dependencies
// in the notebook

// connect to Cassandra and make a keyspace and table as explained in the document
// reverseIndex.rdd.saveToCassandra("search", "keywords",  SomeColumns("keyword", "links", "occurences"))
// reverseIndex.rdd.bulkSaveToCassandra("search", "keywords")
// Create tables
session.execute("CREATE KEYSPACE IF NOT EXISTS search WITH REPLICATION =" +
                "{'class': 'SimpleStrategy', 'replication_factor': 1};")
session.execute("CREATE TABLE IF NOT EXISTS search.keywords (keyword text PRIMARY KEY, links list<text>, occurences list<int>);")

reverseIndex
    .write
    .format("org.apache.spark.sql.cassandra")
    .cassandraFormat("keywords", "search")
    .save()


session.close()

lastException = null


Name: java.lang.NoClassDefFoundError
Message: Could not initialize class org.apache.spark.sql.cassandra.DefaultSource$
StackTrace:   at org.apache.spark.sql.cassandra.package$DataFrameWriterWrapper$.cassandraFormat$extension1(package.scala:55)